In [1]:
import os
import glob
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [20]:
loan_acquisition_file = glob.glob(os.path.join("Raw_data/Acquisition*.txt"))

In [22]:
df_from_each_file = (pd.read_csv(f,sep ="|", index_col=None, header=None) for f in loan_acquisition_file)
df = pd.concat(df_from_each_file, ignore_index=True)

In [23]:
df.rename(columns={
                    0: 'Loan_ID', 1: 'Channel', 2: 'Seller', 3: 'Interest_Rate', 4: 'UPB', 5: 'Loan_Term', 
                    6: 'Origination_Date', 7: 'First_Payment_Date', 8: 'LTV', 9: 'CLTV', 10: 'Num_Borrowers',
                    11: 'DTI', 12: 'Borrower_FICO', 13: 'First_Time_Buyer', 14: 'Loan_Purpose', 15: 'Dwelling_Type', 
                    16: 'Unit_Count', 17: 'Occupancy', 18: 'State', 19: 'Zip', 20: 'Insurance%', 21: 'Product',
                    22: 'Co_Borrower_FICO', 23: 'Mortgage_Insurance_Type',24: 'Relocation_Indicator'}, 
                    inplace=True)

In [24]:
df.to_csv('Acquisition_Q3_merged.csv',index=False)

In [2]:
loan_performance_file = glob.glob(os.path.join( "Raw_data/Performance_*.txt"))

In [3]:
def processing(df):

    df['MONTHLY  REPORTING  PERIOD'] = df['MONTHLY  REPORTING  PERIOD'].apply(str).str[6:].apply(int)*100+df['MONTHLY  REPORTING  PERIOD'].apply(str).str[:2].apply(int)

    idx = df.groupby(['LOAN IDENTIFIER'])['MONTHLY  REPORTING  PERIOD'].transform(max) == df['MONTHLY  REPORTING  PERIOD']

    df = df[idx].copy()
    
    return df


In [4]:
for file in loan_performance_file:
    cols = ['LOAN IDENTIFIER', 'MONTHLY  REPORTING  PERIOD', 'SERVICER  NAME', 'currIntRate', 'CURRENT  ACTUAL  UPB',
            'LOAN AGE', 'REMAINING  MONTHS  TO  LEGAL MATURITY', 'ADJUSTED  MONTHS  TO  MATURITY', 'MATURITY  DATE', 
            'METROPOLITAN  STATISTICAL  AREA  (MSA)', 'CURRENT  LOAN  DELINQUENCY  STATUS', 'MODIFICATION  FLAG',
            'ZERO BALANCE CODE', 'ZERO  BALANCE  EFFECTIVE  DATE', 'LAST PAID INSTALLMENT  DATE', 'FORECLOSURE  DATE',
            'DISPOSITION  DATE', 'FORECLOSURE  COSTS', 'PROPERTY  PRESERVATION  AND REPAIR  COSTS', 'ASSET  RECOVERY  COSTS',
            'MISCELLANEOUS  HOLDING EXPENSES  AND  CREDITS' , 'ASSOCIATED  TAXES  FOR HOLDING PROPERTY', 'NET  SALE PROCEEDS',
            'CREDIT ENHANCEMENT   PROCEEDS', 'REPURCHASE  MAKE  WHOLE  PROCEEDS', 'OTHER  FORECLOSURE  PROCEEDS', 
            'NON  INTEREST  BEARING  UPB', 'PRINCIPAL FORGIVENESS   AMOUNT', 'REPURCHASE  MAKE  WHOLE  PROCEEDS  FLAG',
            'FORECLOSURE  PRINCIPAL WRITE-OFF  AMOUNT', 'SERVICING  ACTIVITY  INDICATOR']
    
    df = pd.read_csv(file, sep='|', index_col=None, header=None, names=cols)  
    df = processing(df)
    file_name = file.split('.')[0] + '_processed.csv'
    df.to_csv(file_name,index=False)    

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
loan_performance_file = glob.glob(os.path.join( "raw_data/Performance_*.csv"))

In [25]:
df_from_each_file = (pd.read_csv(f) for f in loan_performance_file)
perf_df   = pd.concat(df_from_each_file, ignore_index=True)
perf_df.to_csv('Performace_Q3_merged.csv',index=False)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DtypeWarning: Columns (14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  


In [26]:
perf_merged, acq_merged = pd.read_csv('Performace_Q3_merged.csv'), pd.read_csv('Acquisition_Q3_merged.csv')

In [42]:
loan_data = acq_merged.merge(perf_merged , left_on='Loan_ID', right_on='LOAN IDENTIFIER', how='inner')
loan_data.drop('LOAN IDENTIFIER',inplace=True,axis=1)
loan_data.to_csv('load_data_Q3.csv',index=False)

In [2]:
loan_data = pd.read_csv('RawData/')

FileNotFoundError: [Errno 2] File b'load_data_Q3.csv' does not exist: b'load_data_Q3.csv'

In [46]:
def null_percentage(df):
    '''
    input: dataframe
    output: Null percentage of columns in dataframe
    
    '''
    percent = df.isnull().sum()/len(df.index)*100
    df = pd.DataFrame(round(percent,2),columns=['null_percent'])
    
    
    return df[df['null_percent']!=0.00].sort_values('null_percent',ascending=False)

In [47]:
null_percentage(loan_data)

,null_percent
FORECLOSURE PRINCIPAL WRITE-OFF AMOUNT,100.00
PRINCIPAL FORGIVENESS AMOUNT,99.99
REPURCHASE MAKE WHOLE PROCEEDS,99.86
CREDIT ENHANCEMENT PROCEEDS,99.57
SERVICER NAME,99.50
ASSET RECOVERY COSTS,99.25
NON INTEREST BEARING UPB,99.22
OTHER FORECLOSURE PROCEEDS,99.18
PROPERTY PRESERVATION AND REPAIR COSTS,98.93
ASSOCIATED TAXES FOR HOLDING PROPERTY,98.79


In [48]:
loan_data.columns

Index(['Loan_ID', 'Channel', 'Seller', 'Interest_Rate', 'UPB', 'Loan_Term',
       'Origination_Date', 'First_Payment_Date', 'LTV', 'CLTV',
       'Num_Borrowers', 'DTI', 'Borrower_FICO', 'First_Time_Buyer',
       'Loan_Purpose', 'Dwelling_Type', 'Unit_Count', 'Occupancy', 'State',
       'Zip', 'Insurance%', 'Product', 'Co_Borrower_FICO',
       'Mortgage_Insurance_Type', 'Relocation_Indicator',
       'MONTHLY  REPORTING  PERIOD', 'SERVICER  NAME', 'currIntRate',
       'CURRENT  ACTUAL  UPB', 'LOAN AGE',
       'REMAINING  MONTHS  TO  LEGAL MATURITY',
       'ADJUSTED  MONTHS  TO  MATURITY', 'MATURITY  DATE',
       'METROPOLITAN  STATISTICAL  AREA  (MSA)',
       'CURRENT  LOAN  DELINQUENCY  STATUS', 'MODIFICATION  FLAG',
       'ZERO BALANCE CODE', 'ZERO  BALANCE  EFFECTIVE  DATE',
       'LAST PAID INSTALLMENT  DATE', 'FORECLOSURE  DATE', 'DISPOSITION  DATE',
       'FORECLOSURE  COSTS', 'PROPERTY  PRESERVATION  AND REPAIR  COSTS',
       'ASSET  RECOVERY  COSTS',
       'MISCEL

In [49]:

loan_data['ZERO BALANCE CODE'].fillna(0,inplace=True)

zero_bal_cd_map = {0:'Current',1:'Prepaid',2:'Third Party Sale',3:'Short Sale',
                   6:'Repurchase',9:'REO',15:'Note Sale',16:'RPL Loan Sale'}

loan_data['ZERO BALANCE CODE'] = loan_data['ZERO BALANCE CODE'].map(zero_bal_cd_map).apply(str)

zero_bal_cd_map = {'Current':'Current','Prepaid':'Prepaid','Third Party Sale':'Underperforming','Short Sale':'Underperforming',
                   'Repurchase':'Underperforming','REO':'Underperforming','Note Sale':'Underperforming','RPL Loan Sale':'Underperforming'}

loan_data['Current_Status'] = loan_data['ZERO BALANCE CODE'].map(zero_bal_cd_map).apply(str)

In [50]:
loan_data['Mortgage_Insurance_Type'].fillna(0,inplace=True)
loan_data['Insurance%'].fillna(0,inplace=True)
loan_data['Num_Borrowers'].fillna(1,inplace=True)
loan_data['CLTV'].fillna(loan_data['LTV'],inplace=True)
loan_data.drop('Co_Borrower_FICO',axis=1,inplace=True)

In [51]:
FICO_bins = [0,620,660,700,740,780,850]
FICO_labels = ['0-620', '620-660','660-700','700-740','740-780','780+']
loan_data['FICO_bins'] = pd.cut(loan_data['Borrower_FICO'],bins=FICO_bins,labels=FICO_labels)

Term_bins =[0,180,360]
Term_labels =['<=15 Years','<= 30 Years']
loan_data['Term_bins'] = pd.cut(loan_data['Loan_Term'],bins=Term_bins,labels=Term_labels)

loan_data.drop(['Borrower_FICO','Loan_Term'],inplace=True,axis=1)

In [54]:
loan_data['FORECLOSURE  DATE'].fillna(-1,inplace=True)
loan_data['CURRENT  LOAN  DELINQUENCY  STATUS'] = loan_data['CURRENT  LOAN  DELINQUENCY  STATUS'].replace(np.NaN,-1)
loan_data['CURRENT  LOAN  DELINQUENCY  STATUS'] = loan_data['CURRENT  LOAN  DELINQUENCY  STATUS'].replace('X',-1)
loan_data['CURRENT  LOAN  DELINQUENCY  STATUS'] = loan_data['CURRENT  LOAN  DELINQUENCY  STATUS'].astype('int64')

In [58]:
def code_delinq(value):
    
    if (int(value)<30):
        value = 0
    
    if (int(value) >= 30 and int(value) <= 89):
        value = 1
        
    if (int(value) > 89):
        value = 2
        
    return value


loan_data['CURRENT  LOAN  DELINQUENCY  STATUS'] = loan_data['CURRENT  LOAN  DELINQUENCY  STATUS'].apply(code_delinq)


In [ ]:
CurrentLoanDeliquencyStatus_map = {0:'low',1:'Moderate',2:'High'}

loan_data['CURRENT  LOAN  DELINQUENCY  STATUS'] = loan_data['CURRENT  LOAN  DELINQUENCY  STATUS'].map(CurrentLoanDeliquencyStatus_map).apply(str)

In [62]:
loan_data['CURRENT  LOAN  DELINQUENCY  STATUS'].value_counts()

low         2081316
Moderate        188
High             62
Name: CURRENT  LOAN  DELINQUENCY  STATUS, dtype: int64

In [64]:
loan_data['Current_Status'].value_counts()

Prepaid            1096625
Current             949530
Underperforming      35411
Name: Current_Status, dtype: int64

In [66]:
loan_data['FORECLOSURE  DATE'].value_counts()

-1            2053662
09/01/2010        526
07/01/2010        504
06/01/2010        482
08/01/2010        480
               ...   
01/01/2009          8
03/01/2005          7
02/01/2005          2
01/01/2005          2
12/01/2004          1
Name: FORECLOSURE  DATE, Length: 176, dtype: int64

In [ ]:
zero_bal_cd_map = {0:'Current',1:'Prepaid',2:'Third Party Sale',3:'Short Sale',
                   6:'Repurchase',9:'REO',15:'Note Sale',16:'RPL Loan Sale'}

zero_bal_cd_map_ = {'Current':'Current','Prepaid':'Prepaid','Third Party Sale':'Underperforming','Short Sale':'Underperforming',
                   'Repurchase':'Underperforming','REO':'Underperforming','Note Sale':'Underperforming','RPL Loan Sale':'Underperforming'}

In [67]:
def target(currentstatus,Delinquent_Ind,Foreclosure_Ind):
    if(Foreclosure_Ind!=-1):
        val='High-Risk'
        
    elif (currentstatus=='Current' and Delinquent_Ind=='High') or (currentstatus =='Underperforming'):
        val='High-Risk'
        
    elif (currentstatus=='Current' and Delinquent_Ind=='Moderate') or (currentstatus=='Prepaid'):
        val='Moderate-Risk'
    
    else: 
        val='No-Risk'
    return val

In [68]:
loan_data['target'] = loan_data.apply(lambda row: target(row['Current_Status'], row['CURRENT  LOAN  DELINQUENCY  STATUS'],
                                               row['FORECLOSURE  DATE']), axis=1)


In [70]:
loan_data['target'].value_counts()

Moderate-Risk    1096813
No-Risk           949280
High-Risk          35473
Name: target, dtype: int64

In [71]:
loan_data.to_csv('processed_loan.csv',index=False)